In [1]:
# from common import load_track, GENRES
import sys
import numpy as np
from math import pi
import pickle
from pickle import dump
import os
from optparse import OptionParser
import glob
import numpy as np
import librosa as lbr
import keras.backend as K

TRACK_COUNT = 2000

# To avoid errors during importing librosa.
import matplotlib
matplotlib.use('Agg')


# GENRES = ['dog', 'rooster', 'pig', 'cow', 'frog', 'cat', 'hen',
#         'insects', 'sheep', 'crow']
WINDOW_SIZE = 2048
WINDOW_STRIDE = WINDOW_SIZE // 2
N_MELS = 128
MEL_KWARGS = {
    'n_fft': WINDOW_SIZE,
    'hop_length': WINDOW_STRIDE,
    'n_mels': N_MELS
}

def get_layer_output_function(model, layer_name):
    input = model.get_layer('input').input
    output = model.get_layer(layer_name).output
    f = K.function([input, K.learning_phase()], output)
    return lambda x: f([x, 0]) # learning_phase = 0 means test

def load_track(filename, enforce_shape=None):
    new_input, sample_rate = lbr.load(filename, mono=True, duration=5)
    features = lbr.feature.melspectrogram(new_input, **MEL_KWARGS).T

    if enforce_shape is not None:
        if features.shape[0] < enforce_shape[0]:
            delta_shape = (enforce_shape[0] - features.shape[0],
                    enforce_shape[1])
            features = np.append(features, np.zeros(delta_shape), axis=0)
        elif features.shape[0] > enforce_shape[0]:
            features = features[: enforce_shape[0], :]

    features[features == 0] = 1e-6
    return (np.log(features), float(new_input.shape[0]) / sample_rate)



def get_default_shape(dataset_path):
    tmp_features, _ = load_track(os.path.join(dataset_path,
        '101 - Dog/1-30226-A.ogg'))
    return tmp_features.shape

def collect_data(dataset_path):
    '''
    Collects data from the dataset into a pickle. Computes a Mel-scaled
    power spectrogram for each track.

    :param dataset_path: path to the GTZAN dataset directory
    :returns: triple (x, y, track_paths) where x is a matrix containing
        extracted features, y is a one-hot matrix of genre labels and
        track_paths is a dict of absolute track paths indexed by row indices in
        the x and y matrices
    '''
    default_shape = get_default_shape(dataset_path)
    x = np.zeros((TRACK_COUNT,) + default_shape, dtype=np.float32)
    y = np.zeros((TRACK_COUNT, 50), dtype=np.float32)
    track_paths = {}
    
    sub_dirs = os.listdir(dataset_path)

    for genre_index, genre_name in enumerate(sub_dirs):
        i = 0
        for fn in glob.glob(os.path.join(dataset_path, genre_name, "*.ogg")):
            path = fn
            track_index = genre_index  * (TRACK_COUNT // 50) + i
            x[track_index], _ = load_track(fn, default_shape)
            y[track_index, genre_index] = 1
            track_paths[track_index] = os.path.abspath(path)
            i+=1
            print(fn)
    print(x.shape)
    print(y.shape)
    return (x, y, track_paths)

# if __name__ == '__main__':
#     parser = OptionParser()
#     parser.add_option('-d', '--dataset_path', dest='dataset_path',
#             default=os.path.join(os.path.dirname(__file__), 'data/genres'),
#             help='path to the dataset directory', metavar='DATASET_PATH')
#     parser.add_option('-o', '--output_pkl_path', dest='output_pkl_path',
#             default=os.path.join(os.path.dirname(__file__), 'data/data.pkl'),
#             help='path to the output pickle', metavar='OUTPUT_PKL_PATH')
#     options, args = parser.parse_args()

(x, y, track_paths) = collect_data('data\genres')

data = {'x': x, 'y': y, 'track_paths': track_paths}
    

Using TensorFlow backend.


data\genres\101 - Dog\1-100032-A.ogg
data\genres\101 - Dog\1-110389-A.ogg
data\genres\101 - Dog\1-30226-A.ogg
data\genres\101 - Dog\1-30344-A.ogg
data\genres\101 - Dog\1-32318-A.ogg
data\genres\101 - Dog\1-59513-A.ogg
data\genres\101 - Dog\1-85362-A.ogg
data\genres\101 - Dog\1-97392-A.ogg
data\genres\101 - Dog\2-114280-A.ogg
data\genres\101 - Dog\2-114587-A.ogg
data\genres\101 - Dog\2-116400-A.ogg
data\genres\101 - Dog\2-117271-A.ogg
data\genres\101 - Dog\2-118072-A.ogg
data\genres\101 - Dog\2-118964-A.ogg
data\genres\101 - Dog\2-122104-A.ogg
data\genres\101 - Dog\2-122104-B.ogg
data\genres\101 - Dog\3-136288-A.ogg
data\genres\101 - Dog\3-144028-A.ogg
data\genres\101 - Dog\3-155312-A.ogg
data\genres\101 - Dog\3-157695-A.ogg
data\genres\101 - Dog\3-163459-A.ogg
data\genres\101 - Dog\3-170015-A.ogg
data\genres\101 - Dog\3-180256-A.ogg
data\genres\101 - Dog\3-180977-A.ogg
data\genres\101 - Dog\4-182395-A.ogg
data\genres\101 - Dog\4-183992-A.ogg
data\genres\101 - Dog\4-184575-A.ogg
data\ge

data\genres\106 - Cat\3-95694-A.ogg
data\genres\106 - Cat\3-95695-A.ogg
data\genres\106 - Cat\3-95695-B.ogg
data\genres\106 - Cat\3-95697-A.ogg
data\genres\106 - Cat\3-95698-A.ogg
data\genres\106 - Cat\4-120160-A.ogg
data\genres\106 - Cat\4-133047-A.ogg
data\genres\106 - Cat\4-133047-B.ogg
data\genres\106 - Cat\4-133047-C.ogg
data\genres\106 - Cat\4-149940-A.ogg
data\genres\106 - Cat\4-149940-B.ogg
data\genres\106 - Cat\4-161303-A.ogg
data\genres\106 - Cat\4-161303-B.ogg
data\genres\106 - Cat\5-169983-A.ogg
data\genres\106 - Cat\5-172299-A.ogg
data\genres\106 - Cat\5-172639-A.ogg
data\genres\106 - Cat\5-177614-A.ogg
data\genres\106 - Cat\5-214759-A.ogg
data\genres\106 - Cat\5-214759-B.ogg
data\genres\106 - Cat\5-256452-A.ogg
data\genres\106 - Cat\5-259169-A.ogg
data\genres\107 - Hen\1-18074-A.ogg
data\genres\107 - Hen\1-18074-B.ogg
data\genres\107 - Hen\1-31251-A.ogg
data\genres\107 - Hen\1-31251-B.ogg
data\genres\107 - Hen\1-5996-A.ogg
data\genres\107 - Hen\1-71030-A.ogg
data\genres\1

data\genres\201 - Rain\5-193339-A.ogg
data\genres\201 - Rain\5-194892-A.ogg
data\genres\201 - Rain\5-195710-A.ogg
data\genres\201 - Rain\5-198321-A.ogg
data\genres\201 - Rain\5-202898-A.ogg
data\genres\201 - Rain\5-203739-A.ogg
data\genres\202 - Sea waves\1-28135-A.ogg
data\genres\202 - Sea waves\1-28135-B.ogg
data\genres\202 - Sea waves\1-39901-A.ogg
data\genres\202 - Sea waves\1-39901-B.ogg
data\genres\202 - Sea waves\1-43760-A.ogg
data\genres\202 - Sea waves\1-61252-A.ogg
data\genres\202 - Sea waves\1-91359-A.ogg
data\genres\202 - Sea waves\1-91359-B.ogg
data\genres\202 - Sea waves\2-102852-A.ogg
data\genres\202 - Sea waves\2-124662-A.ogg
data\genres\202 - Sea waves\2-125966-A.ogg
data\genres\202 - Sea waves\2-132157-A.ogg
data\genres\202 - Sea waves\2-132157-B.ogg
data\genres\202 - Sea waves\2-133863-A.ogg
data\genres\202 - Sea waves\2-137162-A.ogg
data\genres\202 - Sea waves\2-155801-A.ogg
data\genres\202 - Sea waves\3-144827-A.ogg
data\genres\202 - Sea waves\3-144827-B.ogg
data\g

data\genres\206 - Water drops\3-164592-A.ogg
data\genres\206 - Water drops\3-164593-A.ogg
data\genres\206 - Water drops\3-164594-A.ogg
data\genres\206 - Water drops\3-164595-A.ogg
data\genres\206 - Water drops\3-166324-A.ogg
data\genres\206 - Water drops\3-166326-A.ogg
data\genres\206 - Water drops\4-168155-A.ogg
data\genres\206 - Water drops\4-174797-A.ogg
data\genres\206 - Water drops\4-188293-A.ogg
data\genres\206 - Water drops\4-188293-B.ogg
data\genres\206 - Water drops\4-197871-A.ogg
data\genres\206 - Water drops\4-212604-A.ogg
data\genres\206 - Water drops\4-212604-B.ogg
data\genres\206 - Water drops\4-212604-C.ogg
data\genres\206 - Water drops\5-241846-A.ogg
data\genres\206 - Water drops\5-254832-A.ogg
data\genres\206 - Water drops\5-254832-B.ogg
data\genres\206 - Water drops\5-257349-A.ogg
data\genres\206 - Water drops\5-259180-A.ogg
data\genres\206 - Water drops\5-261433-A.ogg
data\genres\206 - Water drops\5-261439-A.ogg
data\genres\206 - Water drops\5-262641-A.ogg
data\genre

...

data\genres\310 - Drinking - sipping\3-185456-A.ogg
data\genres\310 - Drinking - sipping\4-188191-A.ogg
data\genres\310 - Drinking - sipping\4-188191-B.ogg
data\genres\310 - Drinking - sipping\4-188191-C.ogg
data\genres\310 - Drinking - sipping\4-188595-A.ogg
data\genres\310 - Drinking - sipping\4-194808-A.ogg
data\genres\310 - Drinking - sipping\4-204202-A.ogg
data\genres\310 - Drinking - sipping\4-210593-A.ogg
data\genres\310 - Drinking - sipping\4-213193-A.ogg
data\genres\310 - Drinking - sipping\5-204114-A.ogg
data\genres\310 - Drinking - sipping\5-207836-A.ogg
data\genres\310 - Drinking - sipping\5-207836-B.ogg
data\genres\310 - Drinking - sipping\5-207836-C.ogg
data\genres\310 - Drinking - sipping\5-207836-D.ogg
data\genres\310 - Drinking - sipping\5-243036-A.ogg
data\genres\310 - Drinking - sipping\5-243635-A.ogg
data\genres\310 - Drinking - sipping\5-259640-A.ogg
data\genres\401 - Door knock\1-101336-A.ogg
data\genres\401 - Door knock\1-103995-A.ogg
data\genres\401 - Door knock

data\genres\404 - Door - wood creaks\5-181458-A.ogg
data\genres\404 - Door - wood creaks\5-187444-A.ogg
data\genres\404 - Door - wood creaks\5-188606-A.ogg
data\genres\404 - Door - wood creaks\5-191497-A.ogg
data\genres\405 - Can opening\1-101404-A.ogg
data\genres\405 - Can opening\1-41615-A.ogg
data\genres\405 - Can opening\1-43764-A.ogg
data\genres\405 - Can opening\1-58846-A.ogg
data\genres\405 - Can opening\1-60676-A.ogg
data\genres\405 - Can opening\1-68670-A.ogg
data\genres\405 - Can opening\1-68734-A.ogg
data\genres\405 - Can opening\1-69165-A.ogg
data\genres\405 - Can opening\2-130245-A.ogg
data\genres\405 - Can opening\2-144031-A.ogg
data\genres\405 - Can opening\2-81112-A.ogg
data\genres\405 - Can opening\2-81190-A.ogg
data\genres\405 - Can opening\2-83667-A.ogg
data\genres\405 - Can opening\2-83688-A.ogg
data\genres\405 - Can opening\2-85471-A.ogg
data\genres\405 - Can opening\2-87282-A.ogg
data\genres\405 - Can opening\3-147342-A.ogg
data\genres\405 - Can opening\3-147343-A

data\genres\409 - Clock tick\2-140147-A.ogg
data\genres\409 - Clock tick\2-141584-A.ogg
data\genres\409 - Clock tick\2-88724-A.ogg
data\genres\409 - Clock tick\3-142593-A.ogg
data\genres\409 - Clock tick\3-143933-A.ogg
data\genres\409 - Clock tick\3-150363-A.ogg
data\genres\409 - Clock tick\3-164688-A.ogg
data\genres\409 - Clock tick\3-170377-A.ogg
data\genres\409 - Clock tick\3-170383-A.ogg
data\genres\409 - Clock tick\3-171012-A.ogg
data\genres\409 - Clock tick\3-171041-A.ogg
data\genres\409 - Clock tick\4-175945-A.ogg
data\genres\409 - Clock tick\4-179984-A.ogg
data\genres\409 - Clock tick\4-181035-A.ogg
data\genres\409 - Clock tick\4-181865-A.ogg
data\genres\409 - Clock tick\4-188033-A.ogg
data\genres\409 - Clock tick\4-191327-A.ogg
data\genres\409 - Clock tick\4-194711-A.ogg
data\genres\409 - Clock tick\4-198965-A.ogg
data\genres\409 - Clock tick\5-201194-A.ogg
data\genres\409 - Clock tick\5-208624-A.ogg
data\genres\409 - Clock tick\5-209698-A.ogg
data\genres\409 - Clock tick\5-20

data\genres\504 - Car horn\1-254507-A.ogg
data\genres\504 - Car horn\1-26176-A.ogg
data\genres\504 - Car horn\1-26177-A.ogg
data\genres\504 - Car horn\1-28808-A.ogg
data\genres\504 - Car horn\2-100648-A.ogg
data\genres\504 - Car horn\2-125520-A.ogg
data\genres\504 - Car horn\2-128465-A.ogg
data\genres\504 - Car horn\2-128465-B.ogg
data\genres\504 - Car horn\2-138465-A.ogg
data\genres\504 - Car horn\2-144137-A.ogg
data\genres\504 - Car horn\2-42101-A.ogg
data\genres\504 - Car horn\2-54086-A.ogg
data\genres\504 - Car horn\3-145577-A.ogg
data\genres\504 - Car horn\3-146697-A.ogg
data\genres\504 - Car horn\3-153057-A.ogg
data\genres\504 - Car horn\3-155130-A.ogg
data\genres\504 - Car horn\3-155234-A.ogg
data\genres\504 - Car horn\3-161010-A.ogg
data\genres\504 - Car horn\3-174840-A.ogg
data\genres\504 - Car horn\3-243726-A.ogg
data\genres\504 - Car horn\4-175845-A.ogg
data\genres\504 - Car horn\4-175846-A.ogg
data\genres\504 - Car horn\4-175855-A.ogg
data\genres\504 - Car horn\4-175856-A.o

data\genres\509 - Fireworks\1-115545-C.ogg
data\genres\509 - Fireworks\1-115546-A.ogg
data\genres\509 - Fireworks\1-160563-A.ogg
data\genres\509 - Fireworks\1-160563-B.ogg
data\genres\509 - Fireworks\1-25777-A.ogg
data\genres\509 - Fireworks\1-25781-A.ogg
data\genres\509 - Fireworks\2-117615-A.ogg
data\genres\509 - Fireworks\2-117615-B.ogg
data\genres\509 - Fireworks\2-117615-C.ogg
data\genres\509 - Fireworks\2-117615-D.ogg
data\genres\509 - Fireworks\2-117615-E.ogg
data\genres\509 - Fireworks\2-117616-A.ogg
data\genres\509 - Fireworks\2-117617-A.ogg
data\genres\509 - Fireworks\2-165801-A.ogg
data\genres\509 - Fireworks\3-119120-A.ogg
data\genres\509 - Fireworks\3-119120-B.ogg
data\genres\509 - Fireworks\3-119120-C.ogg
data\genres\509 - Fireworks\3-119120-D.ogg
data\genres\509 - Fireworks\3-119120-E.ogg
data\genres\509 - Fireworks\3-172881-A.ogg
data\genres\509 - Fireworks\3-172922-A.ogg
data\genres\509 - Fireworks\3-251617-A.ogg
data\genres\509 - Fireworks\4-119647-A.ogg
data\genres\5

In [3]:
from keras.callbacks import Callback
from keras.utils import np_utils
from keras.models import Model
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Input, Dense, Lambda, Dropout, Activation, LSTM, \
        TimeDistributed, Convolution1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
import numpy as np
# import cPickle
from optparse import OptionParser
from sys import stderr, argv
import os

SEED = 42
N_LAYERS = 4
FILTER_LENGTH = 5
CONV_FILTER_COUNT = 256
LSTM_COUNT = 256
BATCH_SIZE = 10
EPOCH_COUNT = 100

x = data['x']
y = data['y']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
    
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

def train_model():
    
#     (x_train, x_val, y_train, y_val) = train_test_split(x, y, test_size=0.3,
#             random_state=SEED)

    print('Building model...')

    n_features = x_train.shape[2]
    input_shape = (None, n_features)
    model_input = Input(input_shape, name='input')
    layer = model_input
    for i in range(N_LAYERS):
        # convolutional layer names are used by extract_filters.py
        layer = Convolution1D(
                nb_filter=CONV_FILTER_COUNT,
                filter_length=FILTER_LENGTH,
                name='convolution_' + str(i + 1)
            )(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling1D(2)(layer)
    layer = LSTM(LSTM_COUNT, return_sequences=True)(layer)
    layer = Dropout(0.8)(layer)
    layer = TimeDistributed(Dense(50))(layer)
    layer = Activation('relu')(layer)
    time_distributed_merge_layer = Lambda(
            function=lambda x: K.mean(x, axis=1), 
            output_shape=lambda shape: (shape[0],) + shape[2:],
            name='output_merged')
    layer = time_distributed_merge_layer(layer)
    layer = Dense(50)(layer)
    model_output = Activation('softmax', name='output_realtime')(layer)
    model = Model(model_input, model_output)
    opt = RMSprop(lr=0.00001)
    model.compile(
            loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

    print('Training...')
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, nb_epoch=EPOCH_COUNT,
              validation_data=(x_val, y_val), verbose=1)

    return model

# if __name__ == '__main__':
#     parser = OptionParser()
#     parser.add_option('-d', '--data_path', dest='data_path',
#             default=os.path.join(os.path.dirname(__file__),
#                 'data/data.pkl'),
#             help='path to the data pickle', metavar='DATA_PATH')
#     parser.add_option('-m', '--model_path', dest='model_path',
#             default=os.path.join(os.path.dirname(__file__),
#                 'models/model.yaml'),
#             help='path to the output model YAML file', metavar='MODEL_PATH')
#     parser.add_option('-w', '--weights_path', dest='weights_path',
#             default=os.path.join(os.path.dirname(__file__),
#                 'models/weights.h5'),
#             help='path to the output model weights hdf5 file',
#             metavar='WEIGHTS_PATH')
#     options, args = parser.parse_args()

#     with open(options.data_path, 'r') as f:
#         data = cPickle.load(f)

model = train_model()

#     with open(options.model_path, 'w') as f:
#         f.write(model.to_yaml())
#     model.save_weights(options.weights_path)

Building model...


C:\Users\lisa_\Anaconda3\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(name="convolution_1", filters=256, kernel_size=5)`
C:\Users\lisa_\Anaconda3\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(name="convolution_2", filters=256, kernel_size=5)`
C:\Users\lisa_\Anaconda3\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(name="convolution_3", filters=256, kernel_size=5)`
C:\Users\lisa_\Anaconda3\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(name="convolution_4", filters=256, kernel_size=5)`


Training...


C:\Users\lisa_\Anaconda3\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:69: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1280 samples, validate on 320 samples
Epoch 1/100
1280/1280 [==============================] - 27s 21ms/step - loss: 3.9463 - acc: 0.0234 - val_loss: 3.8662 - val_acc: 0.0688
Epoch 2/100
1280/1280 [==============================] - 23s 18ms/step - loss: 3.8916 - acc: 0.0313 - val_loss: 3.8353 - val_acc: 0.0531
Epoch 3/100
1280/1280 [==============================] - 22s 17ms/step - loss: 3.8541 - acc: 0.0344 - val_loss: 3.8062 - val_acc: 0.0531
Epoch 4/100
1280/1280 [==============================] - 22s 18ms/step - loss: 3.8345 - acc: 0.0547 - val_loss: 3.7811 - val_acc: 0.0469
Epoch 5/100
1280/1280 [==============================] - 22s 17ms/step - loss: 3.8191 - acc: 0.0469 - val_loss: 3.7592 - val_acc: 0.0531
Epoch 6/100
1280/1280 [==============================] - 22s 17ms/step - loss: 3.7941 - acc: 0.0453 - val_loss: 3.7367 - val_acc: 0.0625
Epoch 7/100
1280/1280 [==============================] - 22s 18ms/step - loss: 3.7673 - acc: 0.0469 - val_loss: 3.7161 - val_acc: 0

1280/1280 [==============================] - 20s 15ms/step - loss: 2.9532 - acc: 0.2313 - val_loss: 3.0620 - val_acc: 0.1813
Epoch 61/100
1280/1280 [==============================] - 19s 15ms/step - loss: 2.9451 - acc: 0.2375 - val_loss: 3.0402 - val_acc: 0.2063
Epoch 62/100
1280/1280 [==============================] - 18s 14ms/step - loss: 2.9236 - acc: 0.2539 - val_loss: 3.0409 - val_acc: 0.2000
Epoch 63/100
1280/1280 [==============================] - 19s 15ms/step - loss: 2.9215 - acc: 0.2469 - val_loss: 3.0348 - val_acc: 0.2125
Epoch 64/100
1280/1280 [==============================] - 18s 14ms/step - loss: 2.9108 - acc: 0.2477 - val_loss: 3.0361 - val_acc: 0.2125
Epoch 65/100
1280/1280 [==============================] - 18s 14ms/step - loss: 2.8790 - acc: 0.2578 - val_loss: 3.0381 - val_acc: 0.2063
Epoch 66/100
1280/1280 [==============================] - 20s 15ms/step - loss: 2.8662 - acc: 0.2570 - val_loss: 2.9967 - val_acc: 0.2063
Epoch 67/100
1280/1280 [=======================

In [4]:
model.evaluate(x_test, y_test, verbose=1)

400/400 [==============================] - 1s 3ms/step


[2.6975118160247802, 0.27750000000000002]